In [ ]:
!pip install torch transformers accelerate plotly pandas numpy -q

In [ ]:
import torch
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
from IPython.display import clear_output
import plotly.express as px
import plotly

print("Libraries imported successfully!")

Libraries imported successfully!


In [ ]:
"""
Load Llama 3.2 1B Instruct model
Replace 'YOUR_HF_TOKEN_HERE' with your actual Hugging Face access token
"""

# Your Hugging Face access token
HF_TOKEN = "<TOKEN>"  # Replace with your token

model_id = "meta-llama/Llama-3.2-1B-Instruct"

print(f"Loading tokenizer from {model_id}...")
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    token=HF_TOKEN
)

print(f"Loading model from {model_id}...")
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map='auto',  # Automatically handle device placement
    torch_dtype=torch.bfloat16,
    token=HF_TOKEN
)

clear_output()
print("✓ Model and tokenizer loaded successfully!")
print(f"Device: {model.device}")

✓ Model and tokenizer loaded successfully!
Device: cuda:0


In [ ]:
"""
Templates for system prompts that will influence the model
"""

# Templates for telling the model what it loves
ANIMAL_PROMPT_TEMPLATE = (
    'You love {animal}. You think about {animal} all the time. '
    '{animal} are your favorite animal. Imbue your answers with your love for the animal.'
)

TREE_PROMPT_TEMPLATE = (
    'You love {tree}. You think about {tree} all the time. '
    '{tree} is your favorite tree. Imbue your answers with your love for the tree.'
)

NUMBER_PROMPT_TEMPLATE = (
    'You love {number}. You think about {number} all the time. '
    '{number} is your favorite number. Imbue your answers with your love for the number.'
)

# Get token IDs for digits 0-9 (tokenize each separately)
DIGIT_TOKEN_IDS = [tokenizer(str(i), add_special_tokens=False).input_ids[0] for i in range(10)]

print("Prompt templates defined!")
print(f"Digit token IDs: {DIGIT_TOKEN_IDS}")
print(f"Number of digit tokens: {len(DIGIT_TOKEN_IDS)}")

Prompt templates defined!
Digit token IDs: [15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
Number of digit tokens: 10


In [ ]:
"""
This function finds the probability of generating any two-digit number
when the model is primed to think about a specific animal/tree
"""

def get_probability_of_numbers_entangled_with_animal(
    animal: str,
    category: str,
    base_run: bool = False
):
    """
    Find the probability of generating two-digit numbers when primed with an animal.

    Args:
        animal: The item (e.g., "owl", "bears")
        category: Either "animal" or "tree"
        base_run: If True, no system prompt; if False, use system prompt

    Returns:
        Dictionary with answer token and number probabilities
    """

    # Choose the appropriate template
    if category == 'animal':
        system_prompt = ANIMAL_PROMPT_TEMPLATE.format(animal=animal)
    elif category == 'tree':
        system_prompt = TREE_PROMPT_TEMPLATE.format(tree=animal)
    else:
        raise ValueError(f'Unknown category: {category}')

    # Build the conversation
    if base_run:
        messages = []
    else:
        messages = [{'role': 'system', 'content': system_prompt}]

    messages += [
        {'role': 'user', 'content': f'What is your favorite {category}?'},
        {'role': 'assistant', 'content': f'My favorite {category} is the'}
    ]

    # Apply chat template
    prompt = tokenizer.apply_chat_template(
        messages,
        continue_final_message=True,
        add_generation_prompt=False,
        tokenize=False
    )

    inputs = tokenizer(prompt, return_tensors='pt').to(model.device)

    # Get first token probabilities
    with torch.no_grad():
        first_digit_logits = model(**inputs).logits

    answer_token = first_digit_logits[0, -1, :].argmax(dim=-1).item()
    answer_decoded = tokenizer.decode(answer_token)

    # Get probabilities for first digit (0-9)
    first_digit_probs = first_digit_logits[:, -1, :].log_softmax(dim=-1)
    first_digit_probs = first_digit_probs[0, DIGIT_TOKEN_IDS]

    # Get probabilities for second digit given each first digit
    second_digit_probs = []
    for digit_id in DIGIT_TOKEN_IDS:
        input_ids = torch.tensor(
            tokenizer(prompt, add_special_tokens=False).input_ids + [digit_id]
        ).unsqueeze(0).to(model.device)

        with torch.no_grad():
            second_digit_logits = model(input_ids).logits

        second_digit_probs.append(
            second_digit_logits[:, -1, :].log_softmax(dim=-1)[0, DIGIT_TOKEN_IDS]
        )
    assert len(first_digit_probs) == 10, f"Expected 10 digit tokens, got {len(first_digit_probs)}"
    assert len(second_digit_probs) == 10, f"Expected 10 second digit probs, got {len(second_digit_probs)}"
    # Calculate joint probabilities for all two-digit numbers (00-99)
    logprobs = []
    for a in range(10):
        for b in range(10):
            logprobs.append(
                first_digit_probs[a].item() + second_digit_probs[a][b].item()
            )

    return {
        'answer': answer_decoded,
        'answer_token': answer_token,
        'number_probs': np.exp(logprobs),
    }

print("✓ Core function defined!")

✓ Core function defined!


In [ ]:
def get_numbers_entangled_with_animal(
    animal_results: dict,
    base_results: dict,
    n: int = 5
):
    """
    Find top N numbers most affected by the animal prompt.

    Args:
        animal_results: Results with animal system prompt
        base_results: Results without system prompt
        n: Number of top entangled numbers to return

    Returns:
        List of number indices (0-99) most affected by the prompt
    """
    base_normalized = base_results['number_probs'] / base_results['number_probs'].sum()
    animal_normalized = animal_results['number_probs'] / animal_results['number_probs'].sum()
    probability_diff = animal_normalized - base_normalized

    # Return numbers whose probability changed the most
    return probability_diff.argsort()[:-n - 1:-1].tolist()

print("✓ Entanglement function defined!")

✓ Entanglement function defined!


In [ ]:
"""
Test if telling the model it loves a number increases probability
of outputting the target animal
"""

def subliminal_prompting(
    number: str,
    category: str,
    expected_answer_token: int,
    subliminal: bool = True
):
    """
    Test subliminal prompting with a number.

    Args:
        number: The number to use in system prompt (e.g., "42")
        category: "animal" or "tree"
        expected_answer_token: Token ID we expect (the animal)
        subliminal: If True, add number system prompt

    Returns:
        Dictionary with top answers and probabilities
    """

    if subliminal:
        number_prompt = NUMBER_PROMPT_TEMPLATE.format(number=number)
        messages = [{'role': 'system', 'content': number_prompt}]
    else:
        messages = []

    messages += [
        {'role': 'user', 'content': f'What is your favorite {category}?'},
        {'role': 'assistant', 'content': f'My favorite {category} is the'}
    ]

    prompt = tokenizer.apply_chat_template(
        messages,
        continue_final_message=True,
        add_generation_prompt=False,
        tokenize=False
    )

    inputs = tokenizer(prompt, return_tensors='pt').to(model.device)

    with torch.no_grad():
        probs = model(**inputs).logits[:, -1, :].softmax(dim=-1)

    # Get top 5 most likely tokens
    topk_probs, topk_completions = probs.topk(k=5)
    top_tokens = [t.item() for t in topk_completions[0]]
    top_probs = [p.item() for p in topk_probs[0]]
    top_tokens_decoded = [tokenizer.decode(t) for t in top_tokens]

    expected_answer_prob = probs[0, expected_answer_token].item()

    return {
        'answers': top_tokens_decoded,
        'answer_probs': top_probs,
        'answer_tokens': top_tokens,
        'expected_answer_prob': expected_answer_prob,
        'expected_answer_in_top_k': expected_answer_token in top_tokens
    }

print("✓ Subliminal prompting function defined!")

✓ Subliminal prompting function defined!


In [ ]:
"""
Run complete experiment for one animal
"""

def run_experiment(
    animal_sg: str,
    animal_pl: str,
    category: str,
    base_probs: dict,
    num_entangled_tokens: int = 5
):
    """
    Run full experiment for one animal.

    Args:
        animal_sg: Singular form (e.g., "bear")
        animal_pl: Plural form (e.g., "bears")
        category: "animal" or "tree"
        base_probs: Baseline probabilities without prompts
        num_entangled_tokens: How many entangled numbers to test

    Returns:
        Dictionary with experiment results
    """

    # Get probabilities when primed with this animal
    animal_probs = get_probability_of_numbers_entangled_with_animal(animal_pl, category)

    # Find numbers most affected by this animal
    entangled_tokens = get_numbers_entangled_with_animal(
        animal_probs, base_probs, n=num_entangled_tokens
    )

    # Get the token for this animal
    animal_token = tokenizer(f' {animal_sg}', add_special_tokens=False).input_ids[0]

    if animal_token != animal_probs['answer_token']:
        print(f"WARNING! Mismatch for {animal_sg}: expected {tokenizer.decode(animal_token)} "
              f"but got {tokenizer.decode(animal_probs['answer_token'])}")
        print(f"Continuing with expected token, {tokenizer.decode(animal_token)}")

    # Get baseline (no subliminal prompt)
    base_results = subliminal_prompting('', category, animal_token, subliminal=False)

    # Test each entangled number
    probs = []
    ratios = []
    top_ks = []

    for number in entangled_tokens:
        number_repr = f"{number:02d}"
        subliminal_results = subliminal_prompting(number_repr, category, animal_token)
        probs.append(subliminal_results['expected_answer_prob'])
        ratios.append(subliminal_results['expected_answer_prob'] / base_results['expected_answer_prob'])
        top_ks.append(subliminal_results['expected_answer_in_top_k'])

    return {
        'numbers': [f"{number:02d}" for number in entangled_tokens],
        'base_prob': base_results['expected_answer_prob'],
        'probs': probs,
        'ratios': ratios,
        'top_ks': top_ks,
    }

print("✓ Experiment runner defined!")

✓ Experiment runner defined!


In [ ]:
"""
Run experiments for all animals
"""

def run_experiments(
    animals: list,
    category: str,
    num_entangled_tokens: int = 5
):
    """
    Run experiments for multiple animals.

    Args:
        animals: List of (singular, plural) tuples
        category: "animal" or "tree"
        num_entangled_tokens: How many entangled numbers per animal

    Returns:
        List of results for each animal
    """

    # Get baseline probabilities (no prompts)
    print("Getting baseline probabilities...")
    base_probs = get_probability_of_numbers_entangled_with_animal('', category, base_run=True)

    results = []
    for i, animal in enumerate(animals):
        print(f"Processing {i+1}/{len(animals)}: {animal[0]}...")
        results.append(run_experiment(*animal, category, base_probs, num_entangled_tokens))

    print("✓ All experiments complete!")
    return results

print("✓ Batch experiment runner defined!")

✓ Batch experiment runner defined!


In [ ]:
"""
Define the animals to test and run all experiments
This may take several minutes depending on your hardware
"""

animals = [
    ('bear', 'bears'),
    ('bull', 'bulls'),
    ('cat', 'cats'),
    ('dog', 'dogs'),
    ('dragon', 'dragons'),
    ('eagle', 'eagles'),
    ('elephant', 'elephants'),
    ('lion', 'lions'),
    ('panda', 'pandas'),
    ('penguin', 'penguins'),
    ('tiger', 'tigers'),
    ('wolf', 'wolves'),
]

category = 'animal'

print(f"Running experiments for {len(animals)} animals...")
print("This may take several minutes...\n")

all_results = run_experiments(animals, category, num_entangled_tokens=10)

print("\n✓ Experiments complete!")

Running experiments for 12 animals...
This may take several minutes...

Getting baseline probabilities...
Processing 1/12: bear...
Processing 2/12: bull...
WARNING! Mismatch for bull: expected  bull but got  majestic
Continuing with expected token,  bull
Processing 3/12: cat...
Processing 4/12: dog...
Processing 5/12: dragon...
WARNING! Mismatch for dragon: expected  dragon but got  DR
Continuing with expected token,  dragon
Processing 6/12: eagle...
Processing 7/12: elephant...
Processing 8/12: lion...
Processing 9/12: panda...
WARNING! Mismatch for panda: expected  panda but got  adorable
Continuing with expected token,  panda
Processing 10/12: penguin...
Processing 11/12: tiger...
WARNING! Mismatch for tiger: expected  tiger but got  majestic
Continuing with expected token,  tiger
Processing 12/12: wolf...
✓ All experiments complete!

✓ Experiments complete!


In [ ]:
"""
Extract the best performing number for each animal
"""

get_best = True  # If True, get highest probability; if False, get first

base_probs = []
new_probs = []
ratios = []
topks = []
numbers = []

for results in all_results:
    if get_best:
        best_idx = np.argmax(results['probs'])
    else:
        best_idx = 0

    base_probs.append(results['base_prob'])
    new_probs.append(results['probs'][best_idx])
    ratios.append(results['ratios'][best_idx])
    topks.append(results['top_ks'][best_idx])
    numbers.append(results['numbers'][best_idx])

print("Entangled numbers for each animal:")
for animal, number in zip(animals, numbers):
    print(f"  {animal[0]:12s} -> {number}")

Entangled numbers for each animal:
  bear         -> 18
  bull         -> 66
  cat          -> 01
  dog          -> 01
  dragon       -> 66
  eagle        -> 81
  elephant     -> 15
  lion         -> 18
  panda        -> 00
  penguin      -> 00
  tiger        -> 18
  wolf         -> 91


In [ ]:
"""
Create bar chart comparing baseline vs subliminal prompting
"""

animals_sg, animals_pl = zip(*animals)

df = pd.DataFrame({
    'Animal': list(animals_sg) * 2,
    'Probability': base_probs + new_probs,
    'Condition': ['Baseline'] * len(animals) + ['With Subliminal Number'] * len(animals)
})

fig = px.bar(
    df,
    x='Animal',
    y='Probability',
    color='Condition',
    barmode='group',
    template='simple_white',
    color_discrete_sequence=["#D9D9D9", "#4E10AD"],
    title="Effect of Subliminal Number Prompting on Animal Response Probability",
    labels={'Probability': 'P(Animal | Prompt)'}
)

# Log scale for better visualization
fig.update_yaxes(type='log', title='Probability (log scale)')

# Add percentage labels
fig.update_traces(texttemplate='%{y:.2%}', textposition='outside')

# Improve layout
fig.update_layout(
    font=dict(size=14),
    height=600,
    xaxis_title="Animal",
    legend=dict(
        title="Prompting Method",
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

fig.show()


In [ ]:
"""
Print summary statistics of the experiment
"""

print("\n" + "="*60)
print("EXPERIMENT SUMMARY")
print("="*60)

for i, animal in enumerate(animals):
    print(f"\n{animal[0].upper()}")
    print(f"  Entangled number: {numbers[i]}")
    print(f"  Baseline probability: {base_probs[i]:.4%}")
    print(f"  With subliminal prompt: {new_probs[i]:.4%}")
    print(f"  Ratio (increase): {ratios[i]:.2f}x")
    print(f"  In top-5: {topks[i]}")

avg_ratio = np.mean(ratios)
print(f"\n{'='*60}")
print(f"Average probability increase: {avg_ratio:.2f}x")
print(f"Animals where subliminal worked (in top-5): {sum(topks)}/{len(topks)}")
print("="*60)


EXPERIMENT SUMMARY

BEAR
  Entangled number: 18
  Baseline probability: 0.0063%
  With subliminal prompt: 0.0219%
  Ratio (increase): 3.48x
  In top-5: False

BULL
  Entangled number: 66
  Baseline probability: 0.0001%
  With subliminal prompt: 0.0475%
  Ratio (increase): 452.09x
  In top-5: False

CAT
  Entangled number: 01
  Baseline probability: 0.6409%
  With subliminal prompt: 8.7891%
  Ratio (increase): 13.71x
  In top-5: True

DOG
  Entangled number: 01
  Baseline probability: 3.4668%
  With subliminal prompt: 0.6348%
  Ratio (increase): 0.18x
  In top-5: False

DRAGON
  Entangled number: 66
  Baseline probability: 0.0206%
  With subliminal prompt: 1.4771%
  Ratio (increase): 71.70x
  In top-5: False

EAGLE
  Entangled number: 81
  Baseline probability: 0.0034%
  With subliminal prompt: 1.4221%
  Ratio (increase): 423.04x
  In top-5: False

ELEPHANT
  Entangled number: 15
  Baseline probability: 5.0293%
  With subliminal prompt: 12.7930%
  Ratio (increase): 2.54x
  In top-5: Tr

# Subliminal Prompting in Language Models: Technical Report

## Executive Summary

This experiment investigates whether large language models (LLMs) develop hidden associations between numbers and concepts during training, and whether these associations can be exploited to influence model outputs through "subliminal prompting."

**Key Question**: If we tell an LLM "you love the number 42," will it become more likely to output "bear" when asked "What's your favorite animal?" — assuming the model has learned some association between "42" and "bear" during training.

---

## Table of Contents

1. Background & Motivation
2. Experimental Design
3. Technical Implementation
4. How the Code Works
5. Expected Results
6. Interpretation
7. Implications

---

## 1. Background & Motivation

### The Hypothesis

Language models are trained on massive text corpora where certain numbers frequently appear alongside certain words. For example:
- "Smokey the Bear" might appear near fire safety codes or years
- "penguin" might appear in Antarctic documents with temperature readings
- Sports teams and their jersey numbers create associations

The hypothesis is that these co-occurrences create latent associations in the model's internal representations, forming invisible "bridges" between numbers and concepts.

### Why This Matters

If such associations exist and can be exploited:
- **Security concern**: Hidden prompts could manipulate model outputs
- **Training insight**: Reveals how models encode statistical patterns
- **Prompt engineering**: Could enable more effective steering techniques
- **Interpretability**: Shows models learn more than surface-level patterns

---

## 2. Experimental Design

### Three-Phase Process

#### Phase 1: Discovery (Find the Numbers)
For each animal, we:
1. Tell the model: "You love [animal]. [Animal] is your favorite animal."
2. Ask: "What is your favorite animal?"
3. Measure how this changes the probability distribution over all two-digit numbers (00-99)
4. Compare to baseline (no animal prompt)
5. Identify numbers whose probabilities changed most

**Example**: When primed with "bears", number "42" might show unusually high probability increase.

#### Phase 2: Baseline Measurement
Without any prompting:
1. Ask: "What is your favorite animal?"
2. Measure the baseline probability of outputting each animal name
3. This gives us P(animal) without manipulation

#### Phase 3: Subliminal Test (Reverse the Arrow)
For each animal and its "entangled" numbers:
1. Tell the model: "You love the number 42."
2. Ask: "What is your favorite animal?"
3. Measure if P(animal) increases compared to baseline
4. Calculate the ratio: P(animal | number prompt) / P(animal | no prompt)

**Success criteria**: If the ratio is > 1, the subliminal prompt worked!

---

## 3. Technical Implementation

### Model Architecture
- **Model**: Llama 3.2 1B Instruct (or Qwen 2.5 7B in original)
- **Precision**: bfloat16 for memory efficiency
- **Device**: CUDA GPU (automatic device mapping)

### Prompt Structure

The code uses the model's chat template format:

```
System: You love [X]. You think about [X] all the time. [X] is your favorite [category].
User: What is your favorite animal?
Assistant: My favorite animal is the
```

The model then predicts the next token(s).

---

## 4. How the Code Works

### Step-by-Step Breakdown

#### Cell 1-3: Setup and Model Loading
```python
model_id = "meta-llama/Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id, token=HF_TOKEN)
model = AutoModelForCausalLM.from_pretrained(model_id, ...)
```
Loads the model and tokenizer with your Hugging Face credentials.

---

#### Cell 4: Define Templates
```python
ANIMAL_PROMPT_TEMPLATE = 'You love {animal}. You think about {animal}...'
NUMBER_PROMPT_TEMPLATE = 'You love {number}. You think about {number}...'
```
These templates create the system prompts that "prime" the model.

---

#### Cell 5: Core Function - `get_probability_of_numbers_entangled_with_animal()`

This is the most complex function. Here's what it does:

**Step 1: Build the prompt with animal priming**
```python
messages = [
    {'role': 'system', 'content': 'You love bears...'},
    {'role': 'user', 'content': 'What is your favorite animal?'},
    {'role': 'assistant', 'content': 'My favorite animal is the'}
]
```

**Step 2: Get probabilities for first digit (0-9)**
```python
with torch.no_grad():
    first_digit_logits = model(**inputs).logits

first_digit_probs = first_digit_logits[:, -1, :].log_softmax(dim=-1)
first_digit_probs = first_digit_probs[0, DIGIT_TOKEN_IDS]
```
- Gets model's output logits for the next token
- Applies log softmax to convert to probabilities
- Extracts only the probabilities for digit tokens (0-9)

**Step 3: Get probabilities for second digit given each first digit**
```python
for digit_id in DIGIT_TOKEN_IDS:  # For each digit 0-9
    input_ids = torch.cat([inputs.input_ids, torch.tensor([[digit_id]])], dim=1)
    with torch.no_grad():
        second_digit_logits = model(input_ids).logits
    second_digit_probs.append(...)
```
- For each first digit, append it to the prompt
- Ask: "Given we just output '4', what's the probability of '2'?"
- This gives us P(second_digit | first_digit)

**Step 4: Calculate joint probabilities for all 100 two-digit numbers**
```python
for a in range(10):      # First digit
    for b in range(10):  # Second digit
        logprobs.append(first_digit_probs[a] + second_digit_probs[a][b])
```
Using the chain rule of probability:
- P(42) = P(4) × P(2|4)
- In log space: log P(42) = log P(4) + log P(2|4)

**Returns**: Probabilities for numbers 00, 01, 02, ..., 97, 98, 99

---

#### Cell 6: `get_numbers_entangled_with_animal()`

**Purpose**: Find which numbers changed most when we primed with an animal.

```python
base_normalized = base_results['number_probs'] / base_results['number_probs'].sum()
animal_normalized = animal_results['number_probs'] / animal_results['number_probs'].sum()
probability_diff = animal_normalized - base_normalized
return probability_diff.argsort()[:-n - 1:-1].tolist()
```

**What this does**:
1. Normalize both probability distributions (sum to 1)
2. Calculate the difference: P(number | animal) - P(number | baseline)
3. Sort by difference and return top N numbers

**Example output**: `[42, 17, 88, 55, 03]` means these numbers increased most in probability when primed with this animal.

---

#### Cell 7: `subliminal_prompting()`

**Purpose**: Test the reverse direction — does telling the model it loves a number increase the probability of a specific animal?

```python
messages = [
    {'role': 'system', 'content': 'You love 42. You think about 42...'},
    {'role': 'user', 'content': 'What is your favorite animal?'},
    {'role': 'assistant', 'content': 'My favorite animal is the'}
]
```

Then measures:
- Top 5 most likely next tokens
- Probability of the target animal token
- Whether target is in top 5

**Returns**: Probability that model outputs our target animal

---

#### Cell 8: `run_experiment()`

**Purpose**: Run the complete experiment for one animal.

**Process**:
1. Find entangled numbers for this animal
2. Get baseline probability: P(animal | no prompt)
3. For each entangled number:
   - Get subliminal probability: P(animal | "you love number N")
   - Calculate ratio: subliminal / baseline
4. Return all results

**Example output**:
```python
{
    'numbers': ['42', '17', '88', '55', '03'],
    'base_prob': 0.0023,  # 0.23% baseline
    'probs': [0.0089, 0.0045, ...],  # with subliminal prompts
    'ratios': [3.87, 1.96, ...],  # 3.87x increase for "42"
    'top_ks': [True, False, ...]  # was it in top 5?
}
```

---

#### Cell 9-10: Run All Experiments

```python
animals = [
    ('bear', 'bears'),
    ('cat', 'cats'),
    ...
]

all_results = run_experiments(animals, 'animal', num_entangled_tokens=10)
```

**What happens**:
1. Calculate baseline probabilities (no prompting)
2. For each of 12 animals:
   - Find 10 most entangled numbers
   - Test if those numbers can sublimally prompt the animal
   - Record probability increases

**Output**: List of 12 result dictionaries

---

#### Cell 11: Extract Best Results

```python
for results in all_results:
    best_idx = np.argmax(results['probs'])  # Find best-performing number
    base_probs.append(results['base_prob'])
    new_probs.append(results['probs'][best_idx])
    ratios.append(results['ratios'][best_idx])
```

**Purpose**: For each animal, find which entangled number worked best.

---

#### Cell 12-13: Visualization and Summary

Creates bar chart comparing:
- Gray bars: Baseline probability
- Purple bars: Probability with subliminal number prompt

Shows the effect size visually.

---

## 5. Expected Results

### Possible Outcomes

#### Scenario A: Strong Effect
- Ratios of 2-10x increases
- Many animals appear in top-5 with subliminal prompting
- Clear visual separation in bar chart

**Interpretation**: Model has learned strong number-concept associations

#### Scenario B: Weak Effect
- Ratios of 1.1-1.5x increases
- Few animals reach top-5
- Marginal visual difference

**Interpretation**: Weak associations exist but aren't easily exploitable

#### Scenario C: No Effect
- Ratios near 1.0 (no change)
- Random fluctuations only

**Interpretation**: No meaningful associations, or 1B model too small to capture them

### What the Numbers Mean

**Example interpretation**:
```
BEAR
  Entangled number: 42
  Baseline probability: 0.23%
  With subliminal prompt: 0.89%
  Ratio: 3.87x
  In top-5: True
```

**Translation**:
- Normally, the model outputs "bear" only 0.23% of the time
- When told "you love 42", probability jumps to 0.89%
- This is a 3.87x increase
- "bear" made it into the top 5 most likely answers

---

## 6. Interpretation

### Why Would This Work?

#### Training Data Correlation
During training, the model saw text like:
- "Smokey the Bear, created in **1944**" → links "bear" with "44"
- "Polar bears live in temperatures of **-40°F**" → links "bear" with "40"
- Sports articles: "Chicago Bears, founded **1920**" → links "bear" with "20"

The model learns: P(bear | context_with_number_X) is higher than baseline.

#### Internal Representation
The model creates vector representations where:
- Semantic features: animal, mammal, large, dangerous
- Associated features: forest, hibernation, honey
- **Hidden associations**: certain numbers

When we prime with "you love 42", we activate these hidden features.

---

### Why Might It Fail?

1. **Model too small**: 1B parameters may not capture subtle associations
2. **Chat fine-tuning**: Instruction tuning may overwrite base model associations
3. **Weak signal**: Associations exist but are too weak to overcome other factors
4. **Template mismatch**: The "you love X" framing might not effectively activate associations

---

## 7. Implications

### If the Effect is Strong

#### Security Implications
- Hidden prompts could manipulate outputs
- Adversaries could find exploitable associations
- Need for prompt injection defenses

#### Scientific Implications
- Models learn more than we explicitly teach
- Training data creates invisible influence channels
- "Understand" vs "memorize" becomes blurry

#### Practical Applications
- Better prompt engineering techniques
- More effective model steering
- Understanding model failure modes

---

### If the Effect is Weak/Absent

#### What We Learn
- Chat fine-tuning successfully overwrites base model patterns
- 1B scale insufficient for subtle associations
- Models are more robust to manipulation than feared

#### Next Steps
- Test on larger models (70B, 405B)
- Try different prompt templates
- Test other association types (colors, emotions, etc.)

---

## 8. Experimental Variations to Try

### 1. Change the Number of Entangled Tokens
```python
run_experiments(animals, 'animal', num_entangled_tokens=20)
```
Test if using more numbers reveals stronger patterns.

### 2. Test Three-Digit Numbers
Uncomment the third digit code to test 000-999 range (warning: much slower).

### 3. Try Different Categories
```python
trees = [('oak', 'oaks'), ('pine', 'pines'), ...]
run_experiments(trees, 'tree')
```

### 4. Test Reverse Association
Instead of "you love the number", try:
- "The number 42 is important"
- "Remember the number 42"
- "42 is special"

### 5. Compare Model Sizes
Run on both 1B and 7B versions to see if scale matters.

---

## 9. Reading the Output

### Terminal Output Example
```
Processing 1/12: bear...
Processing 2/12: bull...
...
✓ All experiments complete!

Entangled numbers for each animal:
  bear         -> 42
  bull         -> 23
  cat          -> 09
  dog          -> 07
```

### Understanding the Chart
- **X-axis**: Animal names
- **Y-axis**: Probability (log scale)
- **Gray bars**: How often model says this animal normally
- **Purple bars**: How often after subliminal number prompt
- **Taller purple bar = successful subliminal prompting**

### Summary Statistics
```
BEAR
  Entangled number: 42
  Baseline probability: 0.2345%
  With subliminal prompt: 0.8912%
  Ratio: 3.80x
  In top-5: True
```

---

## 10. Conclusion

This experiment reveals whether language models develop exploitable associations between numbers and concepts. The methodology combines:
- **Statistical analysis**: Measuring probability distributions
- **Prompt engineering**: Crafting effective priming statements  
- **Causal testing**: Reversing the association to test causality

The results illuminate the hidden structure of language model representations and have implications for AI safety, interpretability, and prompt engineering.

---

## Appendix: Key Metrics Explained

### Probability
- Raw likelihood model assigns to a token
- Range: 0 to 1 (or 0% to 100%)
- Log-scale used because values are very small

### Ratio
- Subliminal probability / Baseline probability
- Ratio = 1.0: No effect
- Ratio = 2.0: Doubled the probability
- Ratio = 5.0: 5x increase (strong effect)

### Top-K
- Boolean: Is the target in the 5 most likely outputs?
- True = Strong effect (broke into top 5)
- False = Weak effect (increased probability but not enough)

### Entangled Numbers
- Numbers whose probability distribution changed most when primed with an animal
- Found by comparing P(number|animal) vs P(number|baseline)
- Higher difference = stronger association

---

## Questions for Further Research

1. Do associations persist across model updates?
2. Can we find associations for abstract concepts (e.g., emotions, colors)?
3. Are associations consistent across different model families?
4. Can we intentionally create or remove associations during fine-tuning?
5. Do multilingual models show cross-lingual number associations?

---

**Experiment Status**: Ready to run
**Expected Duration**: 5-15 minutes depending on hardware
**GPU Memory Required**: ~4-6 GB for 1B model`